In [19]:
! pip install tensorflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.10.1 requires protobuf<5.0.0dev,>=3.20.1, but you have protobuf 3.19.6 which is incompatible.



     ------------------------------------- 266.3/266.3 MB 10.7 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 439.2/439.2 kB ? eta 0:00:00
     ------------------------------------- 895.9/895.9 kB 55.4 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 32.0 MB/s eta 0:00:00
     --------------------------------------- 14.2/14.2 MB 29.7 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 92.3 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 21.5 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB ? eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ------------------------------------- 781.3/781.3 kB 12.2 MB/s eta 0:00:00
     ---------------------------------------- 151.7/151.7 kB ? eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: 

In [46]:
import tensorflow.compat.v1 as tf
import math
import numpy as np

In [15]:
def __is_bounded(direction,range,index,tokens_leng):
    cover = range*direction
    if cover+index<0 or cover+index >= tokens_leng:
        return True
    else:
        return False

In [4]:
def get_context(tokens, window_size):
    context_pair = []
    for i, token in enumerate(tokens):
        for j in range(1, window_size+1):
            if not __is_bounded(1,j,i,len(tokens)):
                context_pair.append((tokens[i],tokens[i+j]))
            if not __is_bounded(-1,j,i,len(tokens)):
                context_pair.append((tokens[i],tokens[i-j]))
    return context_pair

In [5]:
def __get_word_set(tokens):
    word_set = set()
    for token in tokens:
        word_set.add(token)
    return word_set

In [6]:
def __get_word_index(word_set):
    word_index_dic = dict()
    inverse_word_dic = dict()
    for i,word in enumerate(word_set):
        word_index_dic[word] = i
        inverse_word_dic[i] = word
    return word_index_dic,inverse_word_dic

In [7]:
def generate_batch(context_pair,batch_size):
    batch_list =[]
    batch=[]
    for i,pair in enumerate(context_pair):

        if i %batch_size==0 and i !=0:
            batch_list.append(batch)
            batch = []

        batch.append(pair)
    return batch_list

In [8]:
def get_vec(word,session):
    return session.run(embeddings[word_index_dic[word]])

In [9]:
def __dis(vec1, vec2):
    dis = 0.0
    for i in range(0,len(vec1)):
        dis+=math.pow((vec1[i]-vec2[i]),2)
    return dis

In [10]:
def get_cos_similarity(vec1, vec2):
    vec1_leng=0
    for value in vec1:
        vec1_leng+=(value*value)
    vec1_leng=math.sqrt(vec1_leng)
    vec2_leng=0
    for value in vec2:
        vec2_leng+=(value*value)
    vec2_leng=math.sqrt(vec2_leng)
    product=np.dot(vec1,vec2)

    return product/(vec1_leng*vec2_leng)

In [11]:
def __sim(vec1, vec2):
    return (1 - math.acos(get_cos_similarity(vec1,vec2)) / math.pi)

In [12]:
def one_hot(data, label_size):
    vector = np.zeros((len(data),label_size),dtype='f')
    for i,single in enumerate(data):
        vector[i][single]=1.0
    return vector

In [13]:
def find_cloest_word(word_set,session,target_word):
    sim = 0.0
    vec1 = get_vec(target_word,session)
    result = ''
    for word in word_set:
        if word == target_word:
            continue
        vec2 = get_vec(word,session)
        tmp_sim=__sim(vec1, vec2)
        print('%s : %s : %s' %(target_word,word,tmp_sim))
        if tmp_sim>sim:
            sim = tmp_sim
            result = word
    return result

In [38]:
text =['he is the king','the king is royal','she is the royal queen']
# 看前後文的時候，會看到兩格遠的詞
window_size = 2
# 代表學習到的Vector將會是長度為5的Vector
embedding_size = 5

if __name__ == '__main__':
    context_pair=[]
    word_set = set()

    for sentence in text:
        tokens = sentence.lower().split(' ')
        context_pair += get_context(tokens,window_size)
        tmp_word_set = __get_word_set(tokens)
        for word in tmp_word_set:
            word_set.add(word)
print(word_set)
print("="*50)
print(context_pair)

{'he', 'she', 'king', 'royal', 'queen', 'the', 'is'}
[('he', 'is'), ('he', 'the'), ('is', 'the'), ('is', 'he'), ('is', 'king'), ('the', 'king'), ('the', 'is'), ('the', 'he'), ('king', 'the'), ('king', 'is'), ('the', 'king'), ('the', 'is'), ('king', 'is'), ('king', 'the'), ('king', 'royal'), ('is', 'royal'), ('is', 'king'), ('is', 'the'), ('royal', 'is'), ('royal', 'king'), ('she', 'is'), ('she', 'the'), ('is', 'the'), ('is', 'she'), ('is', 'royal'), ('the', 'royal'), ('the', 'is'), ('the', 'queen'), ('the', 'she'), ('royal', 'queen'), ('royal', 'the'), ('royal', 'is'), ('queen', 'royal'), ('queen', 'the')]


In [40]:
word_index_dic,inverse_word_dic=__get_word_index(word_set)
word_size = len(word_set)
batch_size = len(context_pair)
print(word_index_dic)
print(inverse_word_dic)
print(batch_size)
print(word_size)

{'he': 0, 'she': 1, 'king': 2, 'royal': 3, 'queen': 4, 'the': 5, 'is': 6}
{0: 'he', 1: 'she', 2: 'king', 3: 'royal', 4: 'queen', 5: 'the', 6: 'is'}
34
7


In [41]:
inputs = [word_index_dic[x[0]] for x in context_pair]
labels = [word_index_dic[x[1]] for x in context_pair]
print(inputs)
print(labels)

[0, 0, 6, 6, 6, 5, 5, 5, 2, 2, 5, 5, 2, 2, 2, 6, 6, 6, 3, 3, 1, 1, 6, 6, 6, 5, 5, 5, 5, 3, 3, 3, 4, 4]
[6, 5, 5, 0, 2, 2, 6, 0, 5, 6, 2, 6, 6, 5, 3, 3, 2, 5, 6, 2, 6, 5, 5, 1, 3, 3, 6, 4, 1, 4, 5, 6, 3, 5]


In [49]:
tf.disable_v2_behavior()
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size,])

In [50]:
embeddings = tf.Variable(
    tf.random_uniform([word_size, embedding_size], -1.0, 1.0))

In [52]:
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [53]:
nce_weights = tf.Variable(
    tf.truncated_normal([word_size,embedding_size],
                        stddev=1.0 / math.sqrt(embedding_size)))

nce_biases = tf.Variable(tf.zeros([word_size]))

In [54]:
prediction = tf.add(tf.matmul(embed, tf.transpose(nce_weights)), nce_biases)

In [55]:
train_labels_vector = tf.one_hot(train_labels,word_size)

In [56]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=train_labels_vector))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [57]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)
session = tf.Session()

In [58]:
init = tf.global_variables_initializer()
session.run(init)

In [63]:
init = tf.global_variables_initializer()
session.run(init)

In [65]:
for iteration in range(0,10000):
    total_loss = 0

    feed_dict = {train_inputs: inputs, train_labels: labels}
    _, cur_loss,pred= session.run([optimizer, loss, prediction], feed_dict=feed_dict)
    print('%s: loss: %s' %(iteration,cur_loss))

0: loss: 1.3204559
1: loss: 1.3204561
2: loss: 1.3204561
3: loss: 1.3204563
4: loss: 1.3204564
5: loss: 1.3204565
6: loss: 1.3204565
7: loss: 1.3204566
8: loss: 1.3204567
9: loss: 1.3204567
10: loss: 1.3204569
11: loss: 1.320457
12: loss: 1.3204571
13: loss: 1.3204572
14: loss: 1.3204572
15: loss: 1.3204575
16: loss: 1.3204575
17: loss: 1.3204577
18: loss: 1.3204577
19: loss: 1.3204579
20: loss: 1.3204579
21: loss: 1.3204582
22: loss: 1.3204582
23: loss: 1.3204584
24: loss: 1.3204584
25: loss: 1.3204585
26: loss: 1.3204585
27: loss: 1.3204588
28: loss: 1.3204588
29: loss: 1.320459
30: loss: 1.320459
31: loss: 1.3204592
32: loss: 1.3204595
33: loss: 1.3204597
34: loss: 1.3204598
35: loss: 1.32046
36: loss: 1.32046
37: loss: 1.3204602
38: loss: 1.3204603
39: loss: 1.3204606
40: loss: 1.3204606
41: loss: 1.3204608
42: loss: 1.3204608
43: loss: 1.3204612
44: loss: 1.3204613
45: loss: 1.3204615
46: loss: 1.3204616
47: loss: 1.320462
48: loss: 1.320462
49: loss: 1.3204622
50: loss: 1.3204622

1102: loss: 1.3205355
1103: loss: 1.3205379
1104: loss: 1.320537
1105: loss: 1.3205395
1106: loss: 1.3205385
1107: loss: 1.3205409
1108: loss: 1.32054
1109: loss: 1.3205425
1110: loss: 1.3205415
1111: loss: 1.3205441
1112: loss: 1.320543
1113: loss: 1.3205458
1114: loss: 1.3205446
1115: loss: 1.3205473
1116: loss: 1.3205462
1117: loss: 1.320549
1118: loss: 1.3205479
1119: loss: 1.3205508
1120: loss: 1.3205494
1121: loss: 1.3205525
1122: loss: 1.3205513
1123: loss: 1.3205543
1124: loss: 1.320553
1125: loss: 1.3205559
1126: loss: 1.3205545
1127: loss: 1.3205576
1128: loss: 1.3205563
1129: loss: 1.3205595
1130: loss: 1.3205581
1131: loss: 1.3205614
1132: loss: 1.3205599
1133: loss: 1.3205632
1134: loss: 1.3205615
1135: loss: 1.3205651
1136: loss: 1.3205634
1137: loss: 1.3205669
1138: loss: 1.3205652
1139: loss: 1.3205688
1140: loss: 1.3205671
1141: loss: 1.3205708
1142: loss: 1.320569
1143: loss: 1.3205729
1144: loss: 1.320571
1145: loss: 1.3205746
1146: loss: 1.3205729
1147: loss: 1.3205

2203: loss: 1.3206043
2204: loss: 1.3206006
2205: loss: 1.3206046
2206: loss: 1.3206009
2207: loss: 1.3206049
2208: loss: 1.3206013
2209: loss: 1.3206053
2210: loss: 1.3206015
2211: loss: 1.3206054
2212: loss: 1.3206017
2213: loss: 1.3206059
2214: loss: 1.320602
2215: loss: 1.3206061
2216: loss: 1.3206024
2217: loss: 1.3206064
2218: loss: 1.3206025
2219: loss: 1.3206066
2220: loss: 1.3206028
2221: loss: 1.3206068
2222: loss: 1.3206031
2223: loss: 1.320607
2224: loss: 1.3206031
2225: loss: 1.3206072
2226: loss: 1.3206034
2227: loss: 1.3206073
2228: loss: 1.3206034
2229: loss: 1.3206073
2230: loss: 1.3206035
2231: loss: 1.3206074
2232: loss: 1.3206035
2233: loss: 1.3206077
2234: loss: 1.3206037
2235: loss: 1.3206077
2236: loss: 1.3206037
2237: loss: 1.3206077
2238: loss: 1.3206037
2239: loss: 1.3206077
2240: loss: 1.3206037
2241: loss: 1.3206079
2242: loss: 1.3206037
2243: loss: 1.3206079
2244: loss: 1.3206038
2245: loss: 1.3206078
2246: loss: 1.3206038
2247: loss: 1.3206078
2248: loss: 

3322: loss: 1.3205568
3323: loss: 1.3205591
3324: loss: 1.3205565
3325: loss: 1.3205588
3326: loss: 1.3205562
3327: loss: 1.3205584
3328: loss: 1.3205558
3329: loss: 1.3205581
3330: loss: 1.3205557
3331: loss: 1.3205576
3332: loss: 1.3205552
3333: loss: 1.3205575
3334: loss: 1.320555
3335: loss: 1.320557
3336: loss: 1.3205545
3337: loss: 1.3205568
3338: loss: 1.3205543
3339: loss: 1.3205564
3340: loss: 1.3205538
3341: loss: 1.320556
3342: loss: 1.3205537
3343: loss: 1.3205557
3344: loss: 1.3205532
3345: loss: 1.3205553
3346: loss: 1.3205531
3347: loss: 1.3205551
3348: loss: 1.3205526
3349: loss: 1.3205545
3350: loss: 1.3205521
3351: loss: 1.3205543
3352: loss: 1.3205519
3353: loss: 1.3205539
3354: loss: 1.3205515
3355: loss: 1.3205535
3356: loss: 1.3205513
3357: loss: 1.3205533
3358: loss: 1.3205508
3359: loss: 1.320553
3360: loss: 1.3205504
3361: loss: 1.3205525
3362: loss: 1.3205501
3363: loss: 1.3205522
3364: loss: 1.3205497
3365: loss: 1.3205519
3366: loss: 1.3205495
3367: loss: 1.

4427: loss: 1.3205338
4428: loss: 1.3205318
4429: loss: 1.3205335
4430: loss: 1.3205316
4431: loss: 1.3205333
4432: loss: 1.3205315
4433: loss: 1.320533
4434: loss: 1.3205312
4435: loss: 1.320533
4436: loss: 1.3205311
4437: loss: 1.3205328
4438: loss: 1.3205309
4439: loss: 1.3205324
4440: loss: 1.3205309
4441: loss: 1.3205323
4442: loss: 1.3205307
4443: loss: 1.3205323
4444: loss: 1.3205305
4445: loss: 1.3205318
4446: loss: 1.3205303
4447: loss: 1.3205318
4448: loss: 1.3205302
4449: loss: 1.3205316
4450: loss: 1.32053
4451: loss: 1.3205314
4452: loss: 1.3205298
4453: loss: 1.3205312
4454: loss: 1.3205296
4455: loss: 1.3205311
4456: loss: 1.3205295
4457: loss: 1.3205309
4458: loss: 1.3205293
4459: loss: 1.3205308
4460: loss: 1.3205291
4461: loss: 1.3205308
4462: loss: 1.320529
4463: loss: 1.3205304
4464: loss: 1.320529
4465: loss: 1.3205303
4466: loss: 1.3205287
4467: loss: 1.32053
4468: loss: 1.3205285
4469: loss: 1.32053
4470: loss: 1.3205283
4471: loss: 1.3205298
4472: loss: 1.320528

5463: loss: 1.3205283
5464: loss: 1.3205266
5465: loss: 1.320528
5466: loss: 1.3205264
5467: loss: 1.320528
5468: loss: 1.3205264
5469: loss: 1.3205279
5470: loss: 1.3205262
5471: loss: 1.3205278
5472: loss: 1.3205262
5473: loss: 1.3205278
5474: loss: 1.3205261
5475: loss: 1.3205277
5476: loss: 1.3205261
5477: loss: 1.3205276
5478: loss: 1.3205259
5479: loss: 1.3205276
5480: loss: 1.3205259
5481: loss: 1.3205273
5482: loss: 1.3205256
5483: loss: 1.3205273
5484: loss: 1.3205256
5485: loss: 1.3205272
5486: loss: 1.3205255
5487: loss: 1.3205271
5488: loss: 1.3205254
5489: loss: 1.320527
5490: loss: 1.3205254
5491: loss: 1.320527
5492: loss: 1.3205252
5493: loss: 1.3205268
5494: loss: 1.3205252
5495: loss: 1.3205267
5496: loss: 1.3205252
5497: loss: 1.3205266
5498: loss: 1.320525
5499: loss: 1.3205264
5500: loss: 1.320525
5501: loss: 1.3205264
5502: loss: 1.320525
5503: loss: 1.3205264
5504: loss: 1.3205248
5505: loss: 1.3205261
5506: loss: 1.3205247
5507: loss: 1.3205261
5508: loss: 1.320

6440: loss: 1.3205254
6441: loss: 1.3205271
6442: loss: 1.3205254
6443: loss: 1.320527
6444: loss: 1.3205253
6445: loss: 1.320527
6446: loss: 1.3205253
6447: loss: 1.320527
6448: loss: 1.3205252
6449: loss: 1.3205267
6450: loss: 1.3205252
6451: loss: 1.3205267
6452: loss: 1.3205252
6453: loss: 1.3205267
6454: loss: 1.320525
6455: loss: 1.3205266
6456: loss: 1.320525
6457: loss: 1.3205265
6458: loss: 1.320525
6459: loss: 1.3205265
6460: loss: 1.3205248
6461: loss: 1.3205265
6462: loss: 1.3205248
6463: loss: 1.3205264
6464: loss: 1.3205248
6465: loss: 1.3205262
6466: loss: 1.3205248
6467: loss: 1.3205262
6468: loss: 1.3205247
6469: loss: 1.3205262
6470: loss: 1.3205246
6471: loss: 1.3205262
6472: loss: 1.3205246
6473: loss: 1.320526
6474: loss: 1.3205245
6475: loss: 1.320526
6476: loss: 1.3205245
6477: loss: 1.320526
6478: loss: 1.3205243
6479: loss: 1.3205259
6480: loss: 1.3205243
6481: loss: 1.3205258
6482: loss: 1.3205243
6483: loss: 1.3205258
6484: loss: 1.3205242
6485: loss: 1.32052

7521: loss: 1.320524
7522: loss: 1.3205228
7523: loss: 1.320524
7524: loss: 1.3205228
7525: loss: 1.3205239
7526: loss: 1.3205225
7527: loss: 1.3205239
7528: loss: 1.3205225
7529: loss: 1.3205239
7530: loss: 1.3205225
7531: loss: 1.3205239
7532: loss: 1.3205225
7533: loss: 1.3205239
7534: loss: 1.3205224
7535: loss: 1.3205237
7536: loss: 1.3205224
7537: loss: 1.3205237
7538: loss: 1.3205224
7539: loss: 1.3205237
7540: loss: 1.3205223
7541: loss: 1.3205236
7542: loss: 1.3205223
7543: loss: 1.3205236
7544: loss: 1.3205223
7545: loss: 1.3205236
7546: loss: 1.3205223
7547: loss: 1.3205236
7548: loss: 1.3205223
7549: loss: 1.3205235
7550: loss: 1.3205222
7551: loss: 1.3205234
7552: loss: 1.3205222
7553: loss: 1.3205234
7554: loss: 1.3205223
7555: loss: 1.3205234
7556: loss: 1.3205222
7557: loss: 1.3205234
7558: loss: 1.3205221
7559: loss: 1.3205234
7560: loss: 1.3205221
7561: loss: 1.3205234
7562: loss: 1.3205221
7563: loss: 1.3205234
7564: loss: 1.320522
7565: loss: 1.3205233
7566: loss: 1

8655: loss: 1.3205223
8656: loss: 1.320521
8657: loss: 1.3205223
8658: loss: 1.3205211
8659: loss: 1.3205223
8660: loss: 1.320521
8661: loss: 1.3205223
8662: loss: 1.320521
8663: loss: 1.3205222
8664: loss: 1.320521
8665: loss: 1.3205222
8666: loss: 1.320521
8667: loss: 1.3205222
8668: loss: 1.320521
8669: loss: 1.3205222
8670: loss: 1.320521
8671: loss: 1.3205222
8672: loss: 1.320521
8673: loss: 1.3205221
8674: loss: 1.3205209
8675: loss: 1.3205222
8676: loss: 1.3205209
8677: loss: 1.3205222
8678: loss: 1.3205209
8679: loss: 1.3205221
8680: loss: 1.320521
8681: loss: 1.3205221
8682: loss: 1.3205209
8683: loss: 1.3205221
8684: loss: 1.3205208
8685: loss: 1.3205221
8686: loss: 1.3205208
8687: loss: 1.3205221
8688: loss: 1.3205208
8689: loss: 1.3205221
8690: loss: 1.3205208
8691: loss: 1.3205221
8692: loss: 1.3205208
8693: loss: 1.320522
8694: loss: 1.3205208
8695: loss: 1.320522
8696: loss: 1.3205209
8697: loss: 1.320522
8698: loss: 1.3205208
8699: loss: 1.3205218
8700: loss: 1.3205208


9822: loss: 1.320521
9823: loss: 1.3205223
9824: loss: 1.320521
9825: loss: 1.3205222
9826: loss: 1.320521
9827: loss: 1.3205223
9828: loss: 1.320521
9829: loss: 1.3205222
9830: loss: 1.3205209
9831: loss: 1.3205222
9832: loss: 1.3205209
9833: loss: 1.3205222
9834: loss: 1.320521
9835: loss: 1.3205222
9836: loss: 1.3205209
9837: loss: 1.3205222
9838: loss: 1.320521
9839: loss: 1.3205222
9840: loss: 1.320521
9841: loss: 1.3205222
9842: loss: 1.3205209
9843: loss: 1.3205222
9844: loss: 1.3205209
9845: loss: 1.3205222
9846: loss: 1.3205208
9847: loss: 1.3205222
9848: loss: 1.3205209
9849: loss: 1.3205222
9850: loss: 1.320521
9851: loss: 1.3205222
9852: loss: 1.3205209
9853: loss: 1.3205222
9854: loss: 1.3205209
9855: loss: 1.3205221
9856: loss: 1.3205208
9857: loss: 1.3205222
9858: loss: 1.3205208
9859: loss: 1.3205222
9860: loss: 1.3205209
9861: loss: 1.3205222
9862: loss: 1.3205209
9863: loss: 1.3205221
9864: loss: 1.3205208
9865: loss: 1.3205221
9866: loss: 1.3205208
9867: loss: 1.3205

In [66]:
print(find_cloest_word(word_set,session,'king'))

king : he : 0.6419137000942748
king : she : 0.6864344145068355
king : royal : 0.4339267256199223
king : queen : 0.7160331550130485
king : the : 0.4376420401973138
king : is : 0.4050389326322631
queen


In [67]:
print(find_cloest_word(word_set, session, 'queen'))

queen : he : 0.48644653386434866
queen : she : 0.509916736743026
queen : king : 0.7160331550130485
queen : royal : 0.4145084788896024
queen : the : 0.34949574908015635
queen : is : 0.6484089904507231
king


In [68]:
print(find_cloest_word(word_set, session, 'royal'))

royal : he : 0.5465503249980881
royal : she : 0.5176169104892023
royal : king : 0.4339267256199223
royal : queen : 0.4145084788896024
royal : the : 0.38599490481686294
royal : is : 0.3399621330905599
he
